In [1]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 19.9 MB/s eta 0:00:00


In [2]:
# Import necessary libraries
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the Pima Indian Diabetes dataset from sklearn
# Note: Scikit-learn's built-in 'load_diabetes' is a regression dataset.
# We will load the actual diabetes dataset from an external source
import pandas as pd

# Load the Pima Indian Diabetes dataset (from UCI repository)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Load the dataset
df = pd.read_csv(url, names=columns)

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
import numpy as np

# Replace zero values with NaN in columns where zero is not a valid value
cols_with_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0, np.nan)

# Impute the missing values with the mean of the respective column
df.fillna(df.mean(), inplace=True)

# Check if there are any remaining missing values
print(df.isnull().sum())


Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [4]:
# Split into features (X) and target (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Split data into training and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Optional: Scale the data for better model performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Check the shape of the data
print(f'Training set shape: {X_train.shape}')
print(f'Test set shape: {X_test.shape}')


Training set shape: (537, 8)
Test set shape: (231, 8)


In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [6]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters


[I 2025-08-21 17:00:35,419] A new study created in memory with name: no-name-370ebf89-b7ce-467f-a1f2-379209f42c97
[I 2025-08-21 17:00:36,654] Trial 0 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 191, 'max_depth': 17}. Best is trial 0 with value: 0.7728119180633147.
[I 2025-08-21 17:00:37,646] Trial 1 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 142, 'max_depth': 12}. Best is trial 0 with value: 0.7728119180633147.
[I 2025-08-21 17:00:38,884] Trial 2 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 188, 'max_depth': 10}. Best is trial 0 with value: 0.7728119180633147.
[I 2025-08-21 17:00:39,687] Trial 3 finished with value: 0.7635009310986964 and parameters: {'n_estimators': 95, 'max_depth': 9}. Best is trial 0 with value: 0.7728119180633147.
[I 2025-08-21 17:00:40,723] Trial 4 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 88, 'max_depth': 9}. Best is trial 0 with value: 0.77281191

In [7]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7802607076350094
Best hyperparameters: {'n_estimators': 71, 'max_depth': 18}


In [8]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


## Samplers in Optuna

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [10]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters

[I 2025-08-21 17:01:25,329] A new study created in memory with name: no-name-ba1ebce2-47f7-40da-a51b-4fa43debfd57
[I 2025-08-21 17:01:25,804] Trial 0 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 68, 'max_depth': 17}. Best is trial 0 with value: 0.7746741154562384.
[I 2025-08-21 17:01:26,378] Trial 1 finished with value: 0.7560521415270017 and parameters: {'n_estimators': 99, 'max_depth': 9}. Best is trial 0 with value: 0.7746741154562384.
[I 2025-08-21 17:01:27,473] Trial 2 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 183, 'max_depth': 14}. Best is trial 0 with value: 0.7746741154562384.
[I 2025-08-21 17:01:27,872] Trial 3 finished with value: 0.7579143389199255 and parameters: {'n_estimators': 67, 'max_depth': 10}. Best is trial 0 with value: 0.7746741154562384.
[I 2025-08-21 17:01:28,896] Trial 4 finished with value: 0.7579143389199254 and parameters: {'n_estimators': 200, 'max_depth': 3}. Best is trial 0 with value: 0.774674115

In [11]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7821229050279329
Best hyperparameters: {'n_estimators': 124, 'max_depth': 18}


In [12]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


In [13]:
search_space = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20]
}

In [14]:
# Create a study and optimize it using GridSampler
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.GridSampler(search_space))
study.optimize(objective)

[I 2025-08-21 17:02:02,830] A new study created in memory with name: no-name-fc8ea5a5-9bf1-4873-b27b-02c8dc64e11b
[I 2025-08-21 17:02:03,571] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 5}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-08-21 17:02:04,438] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 150, 'max_depth': 10}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-08-21 17:02:04,742] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 50, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-08-21 17:02:05,320] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-08-21 17:02:05,907] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 20}. Best is trial 2 with value: 0.772811

In [15]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7746741154562384
Best hyperparameters: {'n_estimators': 50, 'max_depth': 5}


In [16]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


## Optuna Visualizations

In [17]:
# For visualizations
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances

In [18]:
# 1. Optimization History
plot_optimization_history(study).show()

In [19]:
# 2. Parallel Coordinates Plot
plot_parallel_coordinate(study).show()

In [20]:
# 3. Slice Plot
plot_slice(study).show()

In [21]:
# 4. Contour Plot
plot_contour(study).show()

In [22]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

## Optimizing Multiple ML Models

In [23]:
# Importing the required libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [24]:
# Define the objective function for Optuna
def objective(trial):
    # Choose the algorithm to tune
    classifier_name = trial.suggest_categorical('classifier', ['SVM', 'RandomForest', 'GradientBoosting'])

    if classifier_name == 'SVM':
        # SVM hyperparameters
        c = trial.suggest_float('C', 0.1, 100, log=True)
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

        model = SVC(C=c, kernel=kernel, gamma=gamma, random_state=42)

    elif classifier_name == 'RandomForest':
        # Random Forest hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )

    elif classifier_name == 'GradientBoosting':
        # Gradient Boosting hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )

    # Perform cross-validation and return the mean accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
    return score

In [25]:
# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2025-08-21 17:02:16,602] A new study created in memory with name: no-name-7983319d-fd60-42ab-a0ca-01ae7c9935dd
[I 2025-08-21 17:02:17,439] Trial 0 finished with value: 0.7765363128491619 and parameters: {'classifier': 'RandomForest', 'n_estimators': 170, 'max_depth': 8, 'min_samples_split': 10, 'min_samples_leaf': 1, 'bootstrap': False}. Best is trial 0 with value: 0.7765363128491619.
[I 2025-08-21 17:02:18,685] Trial 1 finished with value: 0.7597765363128491 and parameters: {'classifier': 'RandomForest', 'n_estimators': 240, 'max_depth': 12, 'min_samples_split': 3, 'min_samples_leaf': 8, 'bootstrap': True}. Best is trial 0 with value: 0.7765363128491619.
[I 2025-08-21 17:02:19,298] Trial 2 finished with value: 0.7653631284916201 and parameters: {'classifier': 'RandomForest', 'n_estimators': 108, 'max_depth': 9, 'min_samples_split': 10, 'min_samples_leaf': 4, 'bootstrap': True}. Best is trial 0 with value: 0.7765363128491619.
[I 2025-08-21 17:02:19,330] Trial 3 finished with value: 

In [26]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'classifier': 'SVM', 'C': 0.1221382053533163, 'kernel': 'linear', 'gamma': 'scale'}
Best trial accuracy: 0.7895716945996275


In [27]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.776536,2025-08-21 17:02:16.604358,2025-08-21 17:02:17.439541,0 days 00:00:00.835183,NaN,False,RandomForest,NaN,NaN,NaN,8.0,1.0,10.0,170.0,COMPLETE
1,1,0.759777,2025-08-21 17:02:17.440671,2025-08-21 17:02:18.685059,0 days 00:00:01.244388,NaN,True,RandomForest,NaN,NaN,NaN,12.0,8.0,3.0,240.0,COMPLETE
2,2,0.765363,2025-08-21 17:02:18.686014,2025-08-21 17:02:19.298266,0 days 00:00:00.612252,NaN,True,RandomForest,NaN,NaN,NaN,9.0,4.0,10.0,108.0,COMPLETE
3,3,0.694600,2025-08-21 17:02:19.299119,2025-08-21 17:02:19.330410,0 days 00:00:00.031291,9.520620,NaN,SVM,scale,sigmoid,NaN,NaN,NaN,NaN,NaN,COMPLETE
4,4,0.757914,2025-08-21 17:02:19.331242,2025-08-21 17:02:21.005451,0 days 00:00:01.674209,NaN,True,RandomForest,NaN,NaN,NaN,14.0,6.0,7.0,158.0,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.785847,2025-08-21 17:02:49.525706,2025-08-21 17:02:49.554186,0 days 00:00:00.028480,0.272782,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
96,96,0.787709,2025-08-21 17:02:49.554982,2025-08-21 17:02:49.583210,0 days 00:00:00.028228,0.162739,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
97,97,0.789572,2025-08-21 17:02:49.584062,2025-08-21 17:02:49.613490,0 days 00:00:00.029428,0.142846,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
98,98,0.754190,2025-08-21 17:02:49.614309,2025-08-21 17:02:49.654983,0 days 00:00:00.040674,0.341232,NaN,SVM,auto,sigmoid,NaN,NaN,NaN,NaN,NaN,COMPLETE


In [28]:
study.trials_dataframe()['params_classifier'].value_counts()

,count
params_classifier,
SVM,79
RandomForest,12
GradientBoosting,9


In [29]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

,value
params_classifier,
GradientBoosting,0.747362
RandomForest,0.764122
SVM,0.773213


In [30]:
# 1. Optimization History
plot_optimization_history(study).show()

In [31]:
# 3. Slice Plot
plot_slice(study).show()

In [32]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

In [1]:
! pip install optuna-integration[xgboost]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 21.0 MB/s eta 0:00:00


In [2]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import numpy as np

# Load the Iris dataset
X, y = load_iris(return_X_y=True)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function for XGBoost
def objective(trial):
    # Hyperparameter search space
    param = {
        'verbosity': 0,
        'objective': 'multi:softprob',
        'num_class': 3,
        'eval_metric': 'mlogloss',  # Ensure that the eval_metric is specified here
        'booster': 'gbtree',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'n_estimators': 300,
    }

    # Create DMatrix for XGBoost
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Define a pruning callback based on evaluation metrics
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "eval-mlogloss")  # Match the metric name in the evals list

    # Train the model
    bst = xgb.train(
        param,
        dtrain,
        num_boost_round=300,
        evals=[(dtrain, "train"), (dtest, "eval")],  # Ensure the eval datasets and names are specified
        early_stopping_rounds=30,
        callbacks=[pruning_callback]
    )

    # Predict on the test set
    preds = bst.predict(dtest)
    best_preds = [int(np.argmax(line)) for line in preds]

    # Return accuracy as the objective value
    accuracy = accuracy_score(y_test, best_preds)
    return accuracy

# Create a study with pruning
study = optuna.create_study(direction='maximize', pruner=optuna.pruners.SuccessiveHalvingPruner())
study.optimize(objective, n_trials=50)

# Output the best trial
print(f"Best trial: {study.best_trial.params}")
print(f"Best accuracy: {study.best_value}")


[I 2025-08-21 17:23:39,214] A new study created in memory with name: no-name-491f9180-2d55-44b3-a7a2-7a14ce650f05


[0]	train-mlogloss:1.04801	eval-mlogloss:1.04291
[1]	train-mlogloss:0.97346	eval-mlogloss:0.96504
[2]	train-mlogloss:0.90689	eval-mlogloss:0.89553
[3]	train-mlogloss:0.84487	eval-mlogloss:0.83150
[4]	train-mlogloss:0.79027	eval-mlogloss:0.77459
[5]	train-mlogloss:0.73856	eval-mlogloss:0.71848
[6]	train-mlogloss:0.69345	eval-mlogloss:0.67220
[7]	train-mlogloss:0.65233	eval-mlogloss:0.62995
[8]	train-mlogloss:0.61426	eval-mlogloss:0.58906
[9]	train-mlogloss:0.57872	eval-mlogloss:0.55147
[10]	train-mlogloss:0.54658	eval-mlogloss:0.51681
[11]	train-mlogloss:0.51813	eval-mlogloss:0.48435
[12]	train-mlogloss:0.50183	eval-mlogloss:0.46757
[13]	train-mlogloss:0.47734	eval-mlogloss:0.43952
[14]	train-mlogloss:0.46241	eval-mlogloss:0.42417
[15]	train-mlogloss:0.44319	eval-mlogloss:0.40330
[16]	train-mlogloss:0.42925	eval-mlogloss:0.38737
[17]	train-mlogloss:0.41347	eval-mlogloss:0.37099
[18]	train-mlogloss:0.39826	eval-mlogloss:0.35497
[19]	train-mlogloss:0.38622	eval-mlogloss:0.34309
[20]	train

[I 2025-08-21 17:23:51,439] Trial 0 finished with value: 1.0 and parameters: {'lambda': 1.514566127571649e-08, 'alpha': 2.1004295001196977e-05, 'eta': 0.06296533005216298, 'gamma': 6.769287114569828e-05, 'max_depth': 8, 'min_child_weight': 9, 'subsample': 0.7346820986134426, 'colsample_bytree': 0.5675098412361801}. Best is trial 0 with value: 1.0.


[0]	train-mlogloss:0.90984	eval-mlogloss:0.90481
[1]	train-mlogloss:0.76213	eval-mlogloss:0.76029
[2]	train-mlogloss:0.64427	eval-mlogloss:0.63544
[3]	train-mlogloss:0.55167	eval-mlogloss:0.53625
[4]	train-mlogloss:0.47504	eval-mlogloss:0.46381
[5]	train-mlogloss:0.40939	eval-mlogloss:0.39526
[6]	train-mlogloss:0.35523	eval-mlogloss:0.34126
[7]	train-mlogloss:0.31185	eval-mlogloss:0.30021
[8]	train-mlogloss:0.27391	eval-mlogloss:0.26136
[9]	train-mlogloss:0.24212	eval-mlogloss:0.22911
[10]	train-mlogloss:0.21438	eval-mlogloss:0.20349
[11]	train-mlogloss:0.18909	eval-mlogloss:0.17748
[12]	train-mlogloss:0.16901	eval-mlogloss:0.15816
[13]	train-mlogloss:0.15231	eval-mlogloss:0.13936
[14]	train-mlogloss:0.13777	eval-mlogloss:0.12322
[15]	train-mlogloss:0.12392	eval-mlogloss:0.10964
[16]	train-mlogloss:0.11334	eval-mlogloss:0.10019
[17]	train-mlogloss:0.10186	eval-mlogloss:0.09342
[18]	train-mlogloss:0.09255	eval-mlogloss:0.08418
[19]	train-mlogloss:0.08452	eval-mlogloss:0.07735
[20]	train

[I 2025-08-21 17:23:57,682] Trial 1 finished with value: 1.0 and parameters: {'lambda': 0.11786404796302925, 'alpha': 3.18931053420454e-08, 'eta': 0.14634567739357218, 'gamma': 7.954361212322927e-07, 'max_depth': 5, 'min_child_weight': 1, 'subsample': 0.838847738719019, 'colsample_bytree': 0.8299180633775007}. Best is trial 0 with value: 1.0.


...

[0]	train-mlogloss:1.00634	eval-mlogloss:0.99690
[1]	train-mlogloss:0.87752	eval-mlogloss:0.86216
[2]	train-mlogloss:0.77190	eval-mlogloss:0.75160


[I 2025-08-21 17:24:13,685] Trial 48 pruned. Trial was pruned at iteration 2.


[0]	train-mlogloss:1.02453	eval-mlogloss:1.01851
[1]	train-mlogloss:0.95825	eval-mlogloss:0.94859
[2]	train-mlogloss:0.89746	eval-mlogloss:0.88541


[I 2025-08-21 17:24:13,724] Trial 49 pruned. Trial was pruned at iteration 2.


Best trial: {'lambda': 1.514566127571649e-08, 'alpha': 2.1004295001196977e-05, 'eta': 0.06296533005216298, 'gamma': 6.769287114569828e-05, 'max_depth': 8, 'min_child_weight': 9, 'subsample': 0.7346820986134426, 'colsample_bytree': 0.5675098412361801}
Best accuracy: 1.0


In [3]:
from optuna.visualization import plot_intermediate_values

# 1. Plot intermediate values during the trials
plot_intermediate_values(study).show()